In [1]:
%pip install --upgrade --quiet \
    "google-cloud-aiplatform[agent_engines,langchain]" \
    cloudpickle==3.0.0 \
    "pydantic>=2.10" \
    requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
PROJECT_ID = "my-project-agent-32919"
#LOCATION = "us-east1"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://sree-agent-deploy-bucket" 

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

: 

In [ ]:
from vertexai import agent_engines
from vertexai.preview.reasoning_engines import LangchainAgent

In [ ]:

model = "gemini-2.0-flash"

In [ ]:
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date."""
    import requests

    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()

In [ ]:
get_exchange_rate(currency_from="USD", currency_to="SEK")

In [ ]:
agent = LangchainAgent(
    model=model,
    tools=[get_exchange_rate],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

In [ ]:
agent.query(input="What's the exchange rate from US dollars to Swedish currency on 2025/04/07?")

In [ ]:
agent = LangchainAgent(
    model=model,
    tools=[get_exchange_rate],
) 

In [ ]:
remote_agent = agent_engines.create(
    agent,
    requirements=[
        "google-cloud-aiplatform[agent_engines,langchain]",
        "cloudpickle==3.0.0",
        "pydantic>=2.10",
        "requests",
    ],
)

In [ ]:
remote_agent.query(
    input="What's the exchange rate from US dollars to India rupee currency on today?"
)

In [ ]:
remote_agent.resource_name

In [ ]:
remote_agent.delete()